In [1]:
from transformers import SegformerFeatureExtractor, SegformerForSemanticSegmentation
from PIL import Image
import requests

feature_extractor = SegformerFeatureExtractor.from_pretrained("nvidia/mit-b5")
model = SegformerForSemanticSegmentation.from_pretrained("nvidia/mit-b5")

url = "http://images.cocodataset.org/val2017/000000039769.jpg"
image = Image.open(requests.get(url, stream=True).raw)


preprocessor_config.json:   0%|          | 0.00/272 [00:00<?, ?B/s]

/Users/famer.me/Repository/detr-playground/.venv/lib/python3.9/site-packages/transformers/models/segformer/feature_extraction_segformer.py:28: FutureWarning: The class SegformerFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use SegformerImageProcessor instead.
  warnings.warn(
/Users/famer.me/Repository/detr-playground/.venv/lib/python3.9/site-packages/transformers/models/segformer/image_processing_segformer.py:101: FutureWarning: The `reduce_labels` parameter is deprecated and will be removed in a future version. Please use `do_reduce_labels` instead.
  warnings.warn(


config.json:   0%|          | 0.00/70.0k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.linear_c.3.proj.bias', 'decode_head.batch_norm.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.batch_norm.weight', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.classifier.bias', 'decode_head.linear_c.1.proj.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.2.proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [2]:
example_image_url = "https://cdn-media.huggingface.co/Inference-API/Sample-results-on-the-Cityscapes-dataset-The-above-images-show-how-our-method-can-handle.png"
example_image = Image.open(requests.get(example_image_url, stream=True).raw)

In [3]:
import numpy as np
cityscapes_labels = [
    "road", "sidewalk", "building", "wall", "fence", 
    "pole", "traffic light", "traffic sign", "vegetation", "terrain",
    "sky", "person", "rider", "car", "truck", 
    "bus", "train", "motorcycle", "bicycle"
]
colors = np.random.randint(0, 255, size=(len(cityscapes_labels), 3), dtype=np.uint8)

In [4]:
import torch
from torchvision.transforms import functional as F
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np

# image = example_image
image = Image.open("./datasets/Das3300161.jpg")
inputs = feature_extractor(images=image, return_tensors="pt")
outputs = model(**inputs)
logits = outputs.logits  # shape (batch_size, num_labels, height/4, width/4)


# Overlay the mask on the original image
# Convert original image to numpy array and normalize it
image_np = np.array(image) / 255.0

# Apply softmax and argmax to get the most likely class for each pixel
probabilities = torch.nn.functional.softmax(logits, dim=1)
mask = torch.argmax(probabilities, dim=1)

# Resize mask to the size of the original image
mask = F.resize(mask.unsqueeze(1), size=image_np.shape[:2], interpolation=F.InterpolationMode.NEAREST).squeeze()

# Convert mask to numpy array for visualization
mask_np = mask.detach().cpu().numpy()

# Create an RGB image for each class (you might need to customize this part based on the number of classes and desired colors)
# random 14 colors : np.array([[0, 0, 0], ...]) for each class

seg_image = colors[mask_np]

# Overlay (this can be adjusted to change the transparency or the way of overlaying)
combined = 0.7 * image_np + 0.001 * seg_image

# Create a subplot: 1 row, 3 columns
fig, axs = plt.subplots(1, 3, figsize=(15, 5))

# Original image
axs[0].imshow(image_np)
axs[0].set_title('Original Image')
axs[0].axis('off')

# Segmentation mask
axs[1].imshow(seg_image)
axs[1].set_title('Segmentation Mask')
axs[1].axis('off')

# Combined image
axs[2].imshow(combined)
axs[2].set_title('Overlay Image')
axs[2].axis('off')

# filter label in output include in cityscapes_labels
unique_labels = np.unique(mask_np)

# Add the legend to the last subplot, convert colors to 0-1
patches = [mpatches.Patch(color=colors[i]/255.0, label=cityscapes_labels[i]) for i in range(len(cityscapes_labels)) if i in unique_labels]
axs[2].legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)


# Display the subplot
plt.tight_layout()
plt.show()

IndexError: index 607 is out of bounds for axis 0 with size 19

In [31]:
import json
import os

datasets_dir = "/Users/famer.me/Repository/crack-in-asphalt/2023-12-04/Origin/1"

labels = [
	file for file in os.listdir(datasets_dir) if file.endswith(".json")
]

images = [
	file[0:-5] + ".jpg" for file in labels
]

list(zip(labels, images))

labels_json = [
	json.load(open(os.path.join(datasets_dir, label), "r")) for label in labels
]

set([
	shape["label"]
	for label_json in labels_json
	for shape in label_json["shapes"]
])

{'building', 'car', 'crack', 'dog', 'litter', 'pavement', 'truck', 'tyer'}

In [33]:
labels_json[0].keys()

dict_keys(['version', 'flags', 'shapes', 'imagePath', 'imageData', 'imageHeight', 'imageWidth'])